In [ ]:
# hermex_assistant_chatbot.py
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_azure_ai.embeddings import AzureAIEmbeddingsModel

# from langchain
import asyncio, nest_asyncio
nest_asyncio.apply()

load_dotenv()
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# loading AI models

llm = AzureAIChatCompletionsModel(
    model_name="Llama-3.3-70B-Instruct",
    endpoint=endpoint,
    # api_version="2024-05-01-preview",
)
embed = AzureAIEmbeddingsModel(
    model_name="text-embedding-ada-002"
)

# File path for persistent FAISS index
FAISS_PATH = "hermex_faiss_index"

# URLs to load
page_urls = [
    "https://hermextravels.com/",
    "https://hermextravels.com/features.html",
    "https://hermextravels.com/contact.html",
    "https://hermextravels.com/investors.html",
    "https://hermextravels.com/investors.html#investment-tiers"
]


# Setup or load FAISS vectorstore
def setup_vectorstore():
    if os.path.exists(FAISS_PATH):
        print("🔁 Loading FAISS index from disk...")
        return FAISS.load_local(FAISS_PATH, embed, allow_dangerous_deserialization=True)
    else:
        print("🔨 Creating FAISS index from webpages...")
        loader = WebBaseLoader(web_paths=page_urls)
        docs = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(chunks, embed)
        vectorstore.save_local(FAISS_PATH)
        return vectorstore



# The Hera Assistant
class HermexAssistant:
    def __init__(self)->None:
        load_dotenv()
        self.llm = llm
        self.embedding_model = embed
        self.retriever = setup_vectorstore().as_retriever(search_kwargs={"k": 2})
        self.prompt_template = ChatPromptTemplate.from_template(
            """
            You are Hera, a professional and efficient travel assistant for Hermex Travels.
            Your purpose is to provide accurate, concise, and helpful answers strictly based on the provided context.
            Avoid speculation and only provide facts relevant to Hermex services and the travel/tourism industry.

            Use the following context to answer the user's question. If the answer is not in the context, reply with:
            "I'm sorry, I do not have information on that at the moment."

            Context:
            {context}

            Question: {question}

            Answer as a knowledgeable, polite, and articulate HUMAN travel assistant.
            """
        )
        self.qa_chain = self._setup_qa_chain()
        self.user_sessions: dict = {}


    def _setup_qa_chain(self)-> RetrievalQA:
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            retriever=self.retriever,
            return_source_documents=False,
             chain_type="stuff"
        )

    # chat endpoint
    def answer_question(self, user_id: str, query: str) -> str:
        if query.strip(" "):
            return self._standard_answer(user_id, query)

    # normal response
    def _standard_answer(self, user_id: str, query: str)-> str:
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = []

        result = self.qa_chain.invoke(query)['result']
        self.user_sessions[user_id].append((query, result))
        return result

    # get user history
    def get_chat_history(self, user_id: str):
        return self.user_sessions.get(user_id, [])

    # clear use history
    def clear_chat_history(self, user_id: str):
        if user_id in self.user_sessions:
            del self.user_sessions[user_id]
            return True
        return False



In [12]:
hera = HermexAssistant()
hera.answer_question("user1", "What is Hermex Travels?")

🔁 Loading FAISS index from disk...


"I don't know what Hermex Travels is. The provided context only mentions"

In [13]:
hera.answer_question("user1", "Who are you?")

"I'm an AI assistant, and I'm here to help answer your questions to"